<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [45]:
from dotmap import DotMap
import pandas as pd
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC, Comp, Range, todf, report, tuple_keys
from tools.trust_ncg import minimize
import tools.che as che

In [3]:
R=8.314 # J/(mol K)
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c):
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    W_n_vap = Pw * V_vap / (R * c.T)
    n_vap = jnp.concatenate((c.air_n, W_n_vap))
    P = jnp.sum(n_vap) * R * c.T / V_vap
    return (s.P_f - P)**2

s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.air_y * s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = 360


vsc=VSC(v,s)
model_f = vsc.transform(model)
model_f(vsc.x)

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


289480362.3657249

In [4]:
modelf = jax.jit(vsc.transform(model))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0:0, f: 289480362.365714
1:0, f: 202592002.86275265
2:0, f: 71078468.74842988
3:0, f: 1282241.4617060316
4:0, f: 263.1243293157101
5:0, f: 1.1634839085239364e-05
Final results:
f: 1.1634839085239364e-05
x: [365.71923532]


In [5]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


{'T': DeviceArray([365.71923532], dtype=float64)}


In [6]:
def model2(c):
    r=DotMap()
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    r.W_n_vap = Pw * V_vap / (R * c.T)
    r.n_vap = jnp.concatenate((c.air_n, r.W_n_vap))
    P = jnp.sum(r.n_vap) * R * c.T / V_vap
    return ((s.P_f - P)/s.P_f)**2 + (r.W_n_vap - 0.3)**2, r

In [7]:
s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.air_y * s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(v,s)

In [8]:
modelf = jax.jit(vsc.transform(model2))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0:0, f: 0.18134390814615023
1:1, f: 1.4140975873723391e-05
2:1, f: 6.079588663545894e-10
Final results:
f: 2.67104026669521e-18
x: [ 0.84756637 -0.517435  ]


In [9]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


{'T': DeviceArray([370.00563837], dtype=float64),
 'V_tot': DeviceArray([0.01120357], dtype=float64)}


In [10]:
report(model2, vsc.xtoc(x_min))

vector1   vector4                            
                1         1          2           3    4
0 W_n_vap     0.3                                      
  n_vap            0.283185  0.0762421  0.00363057  0.3

In [ ]:
todf(s)

{(0,): {('vector1', '1'): 100000.0}, (1,): {('vector1', '1'): 200000.0}}

vector3                            vector1
               1          2           3           1
air_y       0.78       0.21        0.01            
W_tot                                             1
V_tot                                          0.01
P_i                                          100000
P_f                                          200000
q                                              2000
T_i                                             298
Vw_i                                     0.00100499
Vair_i                                   0.00899501
air_n   0.283185  0.0762421  0.00363057